In [1]:
# Preprocessing the test data code. Concatenate all the gold standard results into one file.
# So long as the input and gold standard files are in the same order then this is safe.
import glob
import shutil

indexes = []
index = 0

# Concatenate all the results into one file
outFileName = "data/Test_Data/Result_Files/mergedGoldStandard.txt"
with open(outFileName, 'w', encoding='utf8') as outfile:
    for filename in glob.glob('data/Test_Data/Gold_Standard_Files/*.txt'):
        if filename == outFileName:
            # don't want to copy the output into the output
            continue
        with open(filename, 'r', encoding='utf8') as readfile:
            for line in readfile:
                # Remove all the empty lines
                if not line.isspace():
                    indexes.append(index)
                    outfile.write(line)
                index += 1
print(indexes)

index = 0
list_index = 0
# Merge all the input files into one file
outFileName = "data/Test_Data/Result_Files/mergedInputFiles.txt"
with open(outFileName, 'w', encoding='utf8') as outfile:
    for filename in glob.glob('data/Test_Data/Input_Files/*.txt'):
        if filename == outFileName:
            # don't want to copy the output into the output
            continue
        with open(filename, 'r', encoding='utf8') as readfile:
            for line in readfile:
                if list_index >= len(indexes):
                    break
                if index == indexes[list_index]:
                    outfile.write(line)
                    list_index += 1
                index += 1

print(index)
# Note that we need to modify the gold standard files slightly so that the values are compressed to a range 0-1.
# As they are currently on a scale from 0-5 we need only divide them by 5. Rounded to 5 decimal places.

indexes = []
index = 0
# Concatenate all the results into one file
outFileName = "data/Training_Data/Result_Files/mergedGoldStandard.txt"
with open(outFileName, 'w', encoding='utf8') as outfile:
    for filename in glob.glob('data/Training_Data/Gold_Standard_Files/*.txt'):
        with open(filename, 'r', encoding='utf8') as readfile:
            for line in readfile:
                # Remove all the empty lines
                if not line.isspace():
                    indexes.append(index)
                    # Convert the string to a double and then round it to the nearest integer
                    val = str(round((float(line) / 5), 5)) + "\n"
                    outfile.write(val)
                index += 1
print(f'Training indexes: {indexes}')

index = 0
list_index = 0
# Merge all the input files into one file
outFileName = "data/Training_Data/Result_Files/mergedInputFiles.txt"
with open(outFileName, 'w', encoding='utf8') as outfile:
    for filename in glob.glob('data/Test_Data/Input_Files/*.txt'):
        with open(filename, 'r', encoding='utf8') as readfile:
            for line in readfile:
                if list_index >= len(indexes):
                    break
                if index == indexes[list_index]:
                    outfile.write(line)
                    list_index += 1
                index += 1
print(index)

print('Done pre-processing training and test data.')

[8, 15, 28, 34, 40, 41, 47, 48, 61, 65, 80, 87, 92, 102, 107, 110, 116, 120, 121, 132, 136, 139, 141, 142, 155, 159, 175, 184, 189, 193, 202, 209, 212, 215, 227, 232, 236, 239, 242, 249, 254, 258, 271, 272, 286, 291, 292, 298, 305, 313, 314, 315, 318, 322, 325, 334, 337, 352, 356, 359, 367, 370, 372, 376, 385, 389, 391, 396, 400, 410, 411, 415, 428, 429, 432, 433, 436, 445, 447, 450, 470, 471, 476, 490, 494, 496, 502, 516, 521, 535, 537, 548, 549, 567, 571, 581, 582, 583, 599, 603, 607, 615, 616, 628, 629, 639, 641, 643, 648, 650, 659, 660, 662, 664, 666, 675, 678, 682, 683, 684, 694, 695, 704, 705, 713, 718, 721, 723, 736, 737, 742, 743, 745, 749, 750, 761, 768, 770, 778, 779, 784, 787, 806, 829, 831, 854, 855, 861, 862, 868, 869, 871, 875, 877, 879, 882, 887, 889, 892, 901, 908, 909, 916, 921, 925, 937, 945, 947, 950, 955, 961, 965, 966, 975, 976, 987, 990, 998, 1012, 1013, 1014, 1046, 1052, 1062, 1063, 1065, 1066, 1072, 1077, 1080, 1088, 1099, 1106, 1107, 1111, 1127, 1135, 1150, 115